In [1]:
from reinforcement_learning_module import LatentSpaceEnv,SimpleDQN
from designing_engineering_constructs import extract_crispr_guides
from tensorflow.python.client import device_lib
from tensorflow.keras import layers
# Training the agent with DQN
#from stable_baseline3 import DQN 
# Importing from REPROGRAM model
import gymnasium as gym
#from gym import spaces
import tensorflow as tf
import cellrank as cr
import scanpy as sc
import pandas as pd
import numpy as np
#import gym



%load_ext autoreload
%autoreload 2

Global seed set to 0
/Users/brendamelano/miniconda3/envs/scvi_env/lib/python3.11/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/Users/brendamelano/miniconda3/envs/scvi_env/lib/python3.11/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


In [2]:
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

TensorFlow version: 2.16.2
Num GPUs Available: 1


In [3]:
tf.debugging.set_log_device_placement(True)

In [ ]:
# Reading in the action_df from svae+. This represents how certain perturbations can modulate certain latent variables
action_df = pd.read_csv('/Users/brendamelano/Desktop/REPROGRAM/action_df.csv', index_col=0)


# Specifying the target point that I am trying to shift cells to based on the class that I built for cluster 5
target_point = np.array([-0.9369193 , -0.447852  , -0.45398346,  1.777766  ,  0.32149878, -0.60258126])


# Creating the latent space environment
env = LatentSpaceEnv(action_df=action_df, target_point=target_point)


# Initialize DQN agent parameters
alpha = 0.3  # Learning rate.. how exactly does this work algorithmically?
gamma = 0.2   # Discount factor .. For future rewards?
epsilon = 0.05  # Exploration rate


# Create DQN agent
dqn_agent = SimpleDQN(observation_space=env.observation_space,
                      action_space=env.action_space,
                      action_df=action_df,  # Pass action_df to the agent
                      alpha=alpha,
                      gamma=gamma,
                      epsilon=epsilon)


# Training loop parameters
num_episodes = 18
max_steps_per_episode = 60


# Training loop
for episode in range(num_episodes):
    
    # Reset the environment at the start of each episode
    state = env.reset()

    # Setting the reward to 0
    total_reward = 0
    
    for step in range(max_steps_per_episode):
        
        # Choose an action
        action = dqn_agent.choose_action(state)
        
        # Take action in the environment
        next_state, reward, done, _ = env.step(action)
        
        # Learn from the experience
        dqn_agent.learn(state, action, reward, next_state, done)
        
        # Update the current state
        state = next_state
        
        # Accumulate the reward
        total_reward += reward
        
        # End the episode if done
        if done:
            break
    
    # Print episode statistics
    print(f"Episode {episode + 1}/{num_episodes}, Total Reward: {total_reward}, Steps: {step + 1}")

print("Training completed.")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


In [3]:
# Test the agent with initial state from cluster 2
initial_state = np.array([0.5738455 ,  0.6803657 ,  0.52349895, -0.96093357, -1.05216   ,
       -0.6411334])

target_state = env.target_point
perturbations = dqn_agent.get_perturbations_to_target(initial_state, target_state)

print("Perturbations to target:", perturbations)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1

In [4]:
# Defining the perturbations to target from the prediction above
number_list = [503, 24, 24, 402, 515, 144, 509, 509, 509, 509, 509, 509, 10, 10, 10, 10, 10, 10, 10, 224, 564, 509, 564, 509, 509, 564, 509, 509, 216, 216, 216, 564, 509, 216, 216, 509, 216, 621, 621, 410, 491, 410, 564, 564, 454, 454, 454, 454, 454, 454]


# Extract unique values from the list
unique_numbers = list(set(number_list))


# Select the rows from action_df based on the unique indices
selected_rows = action_df.iloc[unique_numbers]


# Print the selected rows (14)
print(selected_rows)

             Latent_0  Latent_1  Latent_2  Latent_3  Latent_4  Latent_5
INTS2        0.586403  0.926285  0.123392  0.344440  0.325550 -0.652777
RPS19       -0.649428 -0.465060 -0.375186  1.553235 -0.000231 -0.215098
RBM4         0.012293  0.024018 -0.148396 -0.295388 -0.082797  0.094491
ACTR8       -0.156527 -0.188033 -0.092831  0.005205  0.042706  0.010740
RPL32       -0.107725 -0.241069 -0.139665  0.283800  0.264678 -0.404546
TSR2        -1.061230 -0.346816 -0.276393  1.440419  0.062958 -0.548632
ENO1         0.011491  0.100250 -0.177479 -1.201127  0.224771  0.314871
PIBF1       -0.576393 -0.068671 -0.511934 -0.362162 -0.176071  1.010916
SRP72        0.739823  0.513975  0.031177 -0.795228 -0.277935 -0.447635
RPS4X       -0.831149 -0.517270 -0.378465  1.731631  0.090433 -0.484941
ATP6AP1      0.344657  0.007332 -0.320151 -0.401606 -0.011523  0.330469
POLD2        0.715591  0.040227 -0.277705 -0.499204  0.474314  0.379284
RPS10-NUDT3 -0.742400 -0.546887 -0.441728  1.172710 -0.141962 -0

In [10]:
selected_rows = action_df[action_df.index.isin(unique_numbers)]

# Print the selected rows
print(selected_rows)

NameError: name 'unique_numbers' is not defined

# 

# Designing engineering constructs

In [7]:
result_df = extract_crispr_guides("/Users/brendamelano/Desktop/REPROGRAM/guide_sequences/RPS15A_guides.txt")

In [8]:
result_df

,Gene Name,Guide Sequence
0,RPS15A,GAGACCGAACCTTAGATTGC
1,RPS15A,CCGAACCTTAGATTGCAGGA
2,RPS15A,TACTCACCGGCGCGGAAAGA
3,RPS15A,AGGATGGCGCCGATGGCAGA
4,RPS15A,AGATTGCAGGATGGCGCCGA
